In [87]:
from os import urandom
import hashlib

ENT = 256
format_string = "{0:0"+str(ENT)+"b}"
ENTsie = ENT//8

CSsize = ENT//32

MS = (ENT+CSsize)//11

random_number = urandom(ENTsie)

CS = hashlib.sha256(random_number).digest()

#This is a string binary representation
pre_binary_seed =  format_string.format(int.from_bytes(random_number,"big"),16) + bin(int.from_bytes(CS,"big"))[2:CSsize+2]

binary_seed_list = [pre_binary_seed[i*11:(i+1)*11] for i in range(MS) ]

words = []
with open("english_word.txt", "r", encoding="utf-8") as f:
    for w in f.readlines():
        words.append(w.strip())

mnemonic_words = []
for i in range(MS):
    mnemonic_words.append(words[int(binary_seed_list[i],2)])
    
binary_seed_list
print(mnemonic_words)

['shallow', 'tuna', 'cruel', 'attend', 'improve', 'clip', 'legal', 'shield', 'push', 'kick', 'little', 'bitter', 'practice', 'toward', 'wide', 'topple', 'virus', 'insect', 'tuna', 'crystal', 'gesture', 'artwork', 'small', 'split']


In [88]:
len(mnemonic_words)

24

In [61]:
cs = bin(int.from_bytes(random_number,"big"))

In [53]:
len(bin(int.from_bytes(random_number,"big"))[2:])

127

In [52]:
bin(int.from_bytes(CS,"big"))[2:CSsize+2]

'1000'

In [55]:
bin(int.from_bytes(random_number,"big"))[1:]

'b1011011000010111101100011010001001110000000000000100111101110101101100101010010000101011111111010010001110100011001000111001110'

In [68]:
"{0:0"+str(ENT)+"b}".format(int.from_bytes(random_number,"big"),16)

ValueError: Single '}' encountered in format string

In [69]:
strung = "{0:0"+str(ENT)+"b}"

In [71]:
strung.format(int.from_bytes(random_number,"big"),16)

'00101001001000011100101111101101101011100100001111011010000100011110110001101011110100000010110100011111110010110011011010100010'